In [5]:
import pandas as pd 
import numpy as np

# Pronóstico de demanda

In [7]:
forecast = pd.read_csv ("FORECAST_ERROR.csv") 

**Consideraciones iniciales**

**Date:** se realiza pronóstico diario a excepción de los días domingo.

**Demand y Forescaste**: Las cifras dependen de la unidad de medida de cada producto, es decir, puede ser kilos, unds, bultos, etc. 

**Warehouse:** Bodega desde la que se atendería dicha demanda

In [12]:
forecast.head(5)

,date,warehouse,product_id,demand,forecast
0,2023-07-18,ALQ,1223,2942.500000,2471.0
1,2023-06-02,ALQ,592,0.000000,12.0
2,2023-07-22,ALQ,558,11.144060,10.0
3,2023-07-15,ALQ,970,19.500000,15.0
4,2023-06-26,ALQ,144936,93.963409,122.0


### Análisis preliminar 

In [18]:
np.sort(np.unique(forecast["date"]))

array(['2023-06-01', '2023-06-02', '2023-06-03', '2023-06-05',
       '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09',
       '2023-06-10', '2023-06-12', '2023-06-13', '2023-06-14',
       '2023-06-15', '2023-06-16', '2023-06-17', '2023-06-19',
       '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23',
       '2023-06-24', '2023-06-26', '2023-06-27', '2023-06-28',
       '2023-06-29', '2023-06-30', '2023-07-01', '2023-07-03',
       '2023-07-04', '2023-07-05', '2023-07-06', '2023-07-07',
       '2023-07-08', '2023-07-10', '2023-07-11', '2023-07-12',
       '2023-07-13', '2023-07-14', '2023-07-15', '2023-07-17',
       '2023-07-18', '2023-07-19', '2023-07-20', '2023-07-21',
       '2023-07-22', '2023-07-24', '2023-07-25', '2023-07-26',
       '2023-07-27', '2023-07-28', '2023-07-29', '2023-07-31',
       '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04',
       '2023-08-05', '2023-08-07', '2023-08-08', '2023-08-09',
       '2023-08-10', '2023-08-11', '2023-08-12', '2023-

In [10]:
np.unique(forecast["product_id"])

array([   252,    253,    259,    268,    286,    305,    319,    327,
          345,    349,    355,    379,    381,    382,    396,    408,
          410,    420,    426,    431,    442,    447,    457,    461,
          490,    502,    506,    541,    543,    552,    558,    564,
          579,    580,    585,    586,    592,    593,    603,    617,
          640,    642,    713,    764,    773,    783,    784,    836,
          839,    935,    936,    970,   1142,   1144,   1149,   1211,
         1213,   1217,   1221,   1223,   1266,   1267,   1284,   1298,
         1306,   1310,   1314,   1318,   1348,   1350,   1352,   1356,
         1360,   1364,   1366,   1368,   1370,   1372,   1374,   1376,
         1378,   1382,   1540,   1542,   1546,   1548,   1552,   1554,
         1556,   1558,   1560,   1562,   1563,   1569,   1571,   1573,
         1575,   1893,   1895,   1901,   1905,   2191,   2193,  12028,
        12038,  12060,  12080,  12087,  12146,  12156,  23458,  29244,
      

In [20]:
ordenes_compra = pd.read_csv("PURCHASE_ORDERS-PRICES.csv")

In [26]:
ordenes_compra.head(30)

,warehouse_code,region_code,id,delivery_date,product_id,name,step_unit,supplier_id,price,quantity
0,ALQ,BOG,12818963,2023-08-26,935,Ají Pimiento Estándar Kg,0.5,253,7000.0,14.0
1,ALQ,BOG,12632793,2023-08-05,935,Ají Pimiento Estándar Kg,0.5,253,7500.0,13.0
2,ALQ,BOG,12504845,2023-07-22,935,Ají Pimiento Estándar Kg,0.5,253,7500.0,15.0
3,ALQ,BOG,12568418,2023-07-29,935,Ají Pimiento Estándar Kg,0.5,253,7500.0,12.0
4,ALQ,BOG,12441285,2023-07-15,935,Ají Pimiento Estándar Kg,0.5,253,7500.0,13.0
5,ALQ,BOG,12033375,2023-06-03,935,Ají Pimiento Estándar Kg,0.5,253,7500.0,10.0
6,ALQ,BOG,12757291,2023-08-19,935,Ají Pimiento Estándar Kg,0.5,253,7000.0,14.0
7,ALQ,BOG,12379040,2023-07-08,935,Ají Pimiento Estándar Kg,0.5,253,7500.0,13.0
8,ALQ,BOG,12308615,2023-07-01,935,Ají Pimiento Estándar Kg,0.5,253,7500.0,14.0
9,ALQ,BOG,12103964,2023-06-10,935,Ají Pimiento Estándar Kg,0.5,253,7500.0,13.0


In [25]:
ordenes_compra.groupby(["warehouse_code","region_code"]).sum()

,,id,delivery_date,product_id,name,step_unit,supplier_id,price,quantity
warehouse_code,region_code,,,,,,,,
ALQ,BOG,27197152575,2023-08-262023-08-052023-07-222023-07-292023-0...,194619313,Ají Pimiento Estándar KgAjí Pimiento Estándar ...,2226.5,7067825,4742233.5,1185606.0


In [31]:
np.unique(ordenes_compra["name"])

array(['Acelga Estándar - Atado', 'Aguacate Hass Pintón - Kg',
       'Aguacate Hass Verde - Kg', 'Aguacate Pintón - Kg',
       'Aguacate Verde - Kg', 'Ajo Sin Pelar - Kg',
       'Ají Dulce Estándar Kg', 'Ají Pimiento Estándar Kg',
       'Apio Estándar (Grande) - Atado',
       'Arracacha Pareja (Pequeña) - Kg', 'Banano Urabá Pintón - Kg',
       'Brócoli Estándar - Unidad',
       'Cebolla Cabezona Blanca Limpia Tamaño Mixto - Kg (Tamaño 🏠)',
       'Cebolla Cabezona Blanca Sin Pelar Tamaño Mixto - Desde 1Kg',
       'Cebolla Cabezona Roja Limpia  Tamaño Mixto - Kg (Tamaño 🏠)',
       'Cebolla Cabezona Roja Pelada Grande KAM Estándar - Kg',
       'Cebolla Cabezona Roja Sin Pelar Tamaño Mixto - Desde 2Kg',
       'Cebolla Larga Junca Malla Malla 500g - Unidad (Tamaño 🏠)',
       'Cebolla Larga Junca Pony Estándar - Desde 1Kg',
       'Cebolla Puerro Estándar - Unidad',
       'Cebollín OFF Estándar - Atado (Tamaño 🏠)',
       'Champiñón Blanco Paris 1Kg - Bandeja',
       'Champiñó

In [34]:
ordenes_compra.groupby(["name","step_unit"]).count().head(30)

,,warehouse_code,region_code,id,delivery_date,product_id,supplier_id,price,quantity
name,step_unit,,,,,,,,
Acelga Estándar - Atado,1.0,66,66,66,66,66,66,66,66
Aguacate Hass Pintón - Kg,1.0,11,11,11,11,11,11,11,11
Aguacate Hass Verde - Kg,1.0,16,16,16,16,16,16,16,16
Aguacate Pintón - Kg,1.0,2,2,2,2,2,2,2,2
Aguacate Verde - Kg,1.0,2,2,2,2,2,2,2,2
Ajo Sin Pelar - Kg,0.5,3,3,3,3,3,3,3,3
Ají Dulce Estándar Kg,0.5,13,13,13,13,13,13,13,13
Ají Pimiento Estándar Kg,0.5,12,12,12,12,12,12,12,12
Apio Estándar (Grande) - Atado,1.0,78,78,78,78,78,78,78,78


In [ ]:
# Id : Número de orden de compra
# delivery_date: Fecha de ingreso del producto a bodega

PURCHASE_ORDERS_PRICE.csv	step_unit	Unidad en la que viene el producto. Por ejemplo, para el producto "Ají Pimiento Estándar Kg" el step unit de 0.5 indica que las unidades compradas son de 0.5kg, por lo cual si el quantity es de 14, se compraron en total 7kg.